# Configurações

![alt text](https://github.com/elthonf/plataformas-cognitivas-docker/blob/master/images/capa.png?raw=1)

In [4]:
#@title Parâmetros da solução
#@markdown Informe a quantidade de clientes que devem ser analisados.
qtde_clientes = 4  #@param {type: "slider", min: 1, max: 10}

#@markdown Informe o IP do servidor de predição da sua solução:
my_server_ip = "38.28.76.130" #@param {type:"string"}

#@markdown Informe a porta do servidor de predição da sua solução, se nada foi alterado, deve ser 443:
my_server_port = "443" #@param {type:"string"}

#@markdown Informe o seu nome:
students_name = "Elthon Manhas de Freitas" #@param {type:"string"}

#@markdown Ah, e não se esqueça de executar esta célula!


In [5]:
import requests
import pandas as pd
import json

# Obtenção dos empréstimos pendentes de análise

Vamos agora obter os clientes que estã precisand de empréstimo e exibí-los na tela

In [6]:
url = "https://us-central1-emf-teacher.cloudfunctions.net/function-aulas-getclient?qtde={0}".format(qtde_clientes)
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers)
response_content = response.text.encode('utf8').decode()
clientes = pd.read_json(response_content)

In [7]:
clientes

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade
11530,"Shafee'a, el-Ebrahimi",2993.095335,25.728829,0,1,0,0,0,2
30924,"Juliette, Vasquez",3206.908198,32.260114,0,1,0,0,1,2
49445,"Thaamir, el-Edris",4708.623196,54.360051,1,0,1,0,0,3
18746,"Uriel, Martinez",4277.132457,45.965060,1,0,0,1,0,3


# Predição

O cliente agora será submetido a dois modelos de Machine Learning preparados para a análise.

In [8]:
url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [12]:
response01 = requests.request("POST", url + f"?model=modelo01&aluno=a", headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': [1, 1, 0, 0],
 'proba': [[0.07350118258752814, 0.9264988174124719],
  [0.193987802140286, 0.806012197859714],
  [0.9990246582909653, 0.0009753417090346871],
  [0.9640461266005345, 0.035953873399465555]]}

In [ ]:
df_01 = pd.Series(respostas01['prediction'], index = clientes.index, name="PredicaoML01")
df_01B = pd.Series(respostas01['proba'], index = clientes.index, name="PredicaoML01B")

In [ ]:
response02 = requests.request("POST", url + "?model=modelo02", headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': [-0.010159329576254223,
  0.5917030901875656,
  0.6236547836165044,
  1.0260868020451164]}

In [ ]:
df_02 = pd.Series(respostas02['prediction'], index = clientes.index, name="PredicaoML02")

In [ ]:
pd.DataFrame(clientes.loc[:,"nome"])\
    .merge(right= df_01, left_index=True, right_index=True)\
    .merge(df_01B, left_index=True, right_index=True)\
    .merge(df_02, left_index=True, right_index=True)

,nome,PredicaoML01,PredicaoML01B,PredicaoML02
22672,"Daniel, Wu",0,"[0.9712613311116339, 0.02873866888836604]",-0.010159
3029,"Brian, Wright",1,"[0.10609475169972526, 0.8939052483002747]",0.591703
19240,"Andre, Dave",1,"[0.16488817236745124, 0.8351118276325488]",0.623655
44198,"Alma Ruby, Cordova",1,"[0.006931977891545316, 0.9930680221084547]",1.026087
